In [ ]:
#SEE IF graphs look better with or without "%matplotlib inline" for you;
#%matplotlib inline
import os

import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
from mpl_toolkits.mplot3d import Axes3D
import random

bench_dir = 'bench_py_v005'

size_ag, size_lex, size_mem = 64, 64, 64

#varexam='AcceptanceRate'
varexam='Comprehension'
#varexam='GrammarVar'


mutators = ['ProbitVectorGaussian', 'ProbitVectorUniform',\
            'AdditiveVectorUniform', 'ProbitSingleGaussian',\
            'ProbitSingleUniform', 'AdditiveSingleUniform', 'NoMutation']

#mrspan_start, mrspan_stop, mrspan_points = 0.0, 2.0, 20+1
mrspan_start, mrspan_stop, mrspan_points = 0.0, 1.0, 50+1

mr_seq = np.linspace(mrspan_start, mrspan_stop, num=mrspan_points, endpoint = True)

print("Sampled values: %s", mr_seq)
plt.rcParams['figure.figsize'] = (15,15)

def dumpread(fn):
    print("Reading file %s." % fn)
    rawreadgen = np.genfromtxt(fn, names = True, delimiter=' ', skip_header = 2, invalid_raise = False, encoding = None)
    cols = rawreadgen.dtype.names
    #print("...read <%s> columns." % ', '.join(cols) )
    return rawreadgen

acc_rate_matrix = dict()

for mutator in mutators:
    acc_rate_matrix[mutator] = dict()
    for mr_val in mr_seq:
        #sorry, varying number of decimal values makes for crappy code.
        fname = 'run_%d_%d_%d_mr_{%.2f}_{%s}.txt' % (size_ag, size_lex, size_mem, mr_val, mutator)
        ffname = os.path.join(bench_dir, fname)
        dg = dumpread(ffname)
        #acc_rate_matrix['%.1f' % mr_val] = dg
        acc_rate_matrix[mutator]['%.2f' % mr_val] = dg
    

In [ ]:
#PLOT TIME SERIES
#PLOT Mutation AVERAGED across runs values, by mutator

obsnum = 1000

#percentiles = [5, 95]
percentiles = [0, 100] #min/max

multirun_avg = dict()
multirun_low = dict()
multirun_high = dict()

for mutator in mutators:
    multirun_avg[mutator] = list()
    multirun_low[mutator] = list()
    multirun_high[mutator] = list()
    for obs in range(0, obsnum):
        vals = list( acc_rate_matrix[mutator][k][varexam][obs] for k in acc_rate_matrix[mutator].keys() )
        avg = np.average( vals )
        low = np.percentile( vals, percentiles[0] )
        high = np.percentile( vals, percentiles[1] )
        multirun_avg[mutator].append(avg)
        multirun_low[mutator].append(low)
        multirun_high[mutator].append(high)


In [ ]:
f, axarr = plt.subplots(len(mutators), sharex=True)

axindex = 0
x = range(0, obsnum)
for mutator in mutators:
    #print mutator,multirun_avg[mutator]
    y = multirun_avg[mutator][0:obsnum]
    y_low = multirun_low[mutator][0:obsnum]
    y_high = multirun_high[mutator][0:obsnum]

    axarr[axindex].set_title(varexam)

    axarr[axindex].plot(x,y_low, label = mutator+'_low')
    axarr[axindex].plot(x,y, label = mutator+'_mean')
    axarr[axindex].plot(x,y_high, label = mutator+'_high')
    axarr[axindex].fill_between(x, y_low, y_high, alpha = .2)
    axarr[axindex].legend(loc = 'upper left')
    axindex += 1
    print("Min/Max %s values for mutator %s are %.3f, %.3f" % (varexam, mutator, min(y), max(y)) )

#plt.legend(loc = 'upper left')
plt.tight_layout()
plt.show()


In [ ]:
for mutator in mutators:

    fig = plt.figure()

    ax = fig.add_subplot(111, projection='3d')

    x = list( map(float, sorted(acc_rate_matrix[mutator].keys())) )
    y = range(0, obsnum)

    X, Y = np.meshgrid(x, y)

    zs = np.array( [acc_rate_matrix[mutator]['%0.2f' % (j)][varexam][k] for k in y for j in x] )
    Z = zs.reshape(X.shape)

    ax.plot_surface(X, Y, Z, cmap = 'RdBu_r', alpha = 0.8)

    ax.set_title(mutator)
    ax.set_xlabel(mutator+' Mutation Scale')
    ax.set_ylabel('Iteration')
    ax.set_zlabel(varexam)

    plt.show()

In [ ]:
#PLOT Mutation / Acceptance TODO
"""
x_mr = []
y_ar = []

for mutator in mutators:
for mr_val in mr_seq:
    x_mr.append(mr_val) 
    #y_ar.append(acc_rate_matrix['%.1f' % mr_val]['AcceptanceRate'][obsnum-1])
    y_ar.append(acc_rate_matrix['%.3f' % mr_val][varexam][obsnum-1])

plt.plot(x_mr,y_ar)
plt.show()
"""